# Bigram.py

Bigram 模型是一种简单的语言模型，它基于前一个词（或字符）来预测下一个词（或字符）的概率。具体来说，在给定当前词或字符的情况下，模型通过学习当前字符与下一个字符之间的关系，来预测下一个可能出现的字符。

Bigram 模型会学习如下的字符对：
- "h" -> "e"
- "e" -> "l"
- "l" -> "l"
- "l" -> "o"

训练之后，模型根据前一个字符来预测下一个字符。例如，当输入字符是 "h" 时，模型会预测 "e" 为下一个字符。

Bigram 模型可以扩展为更高阶的 N-gram 模型

Bigram 模型：

"Bi" 在这个词中表示 “2”，即每次考虑两个连续的词或字符来进行建模。Bigram 模型会根据前一个词来预测下一个词。

N-gram 模型：

"N" 在这个词中表示 任意数，即 N 可以是 1、2、3 等等。N-gram 模型表示每次考虑 N 个连续的词或字符来进行建模。

### 导入库

导入了必要的PyTorch模块，包括神经网络模块nn和一些功能性函数F。

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

### 超参数设置

批量大小（batch_size）、上下文长度（block_size）、训练的最大迭代次数（max_iters）、评估间隔（eval_interval）、学习率（learning_rate）、设备（CPU或GPU），以及评估时的迭代次数（eval_iters）

In [2]:
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

### 数据准备

生成字符集的映射表（stoi 和 itos），用于将字符和整数索引相互转换。

把字符转为int型，为了让计算机理解；计算机输出时，再转为字符，让人类理解

stoi,itos可以理解为字典

In [3]:
with open('./ng-video-lecture/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

### 训练和验证数据集的拆分

数据集分为训练集和验证集，训练集为前90%的数据，验证集为后10%的数据。

In [4]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### 数据加载函数
示例

block_size = 3

batch_size = 2

data = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

ix = [2, 4]

x = [[2, 3, 4],[4, 5, 6]]

y = [[3, 4, 5],[5，6, 7]]

y中与x相对应位置的值，为x应该预测的值，前面预测后面的

In [5]:
# 从训练集或者验证集取数据
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # 把数据放到GPU运行
    x, y = x.to(device), y.to(device)
    return x, y

torch.randint：生成在指定范围内均匀分布的随机整数张量。

torch.randint(low, high, size, out=None, dtype=torch.int64, layout=torch.strided, device=None, requires_grad=False)

    low：生成随机整数的下界（包含该值）。
    high：生成随机整数的上界（不包含该值）。
    size：生成的张量的形状，通常是一个元组，表示每个维度的大小。
    out（可选）：输出的张量。
    dtype（可选）：张量的类型，默认是 torch.int64。
    layout（可选）：张量的布局，默认是 torch.strided。
    device（可选）：张量所在的设备（CPU 或 GPU）。
    requires_grad（可选）：如果为 True，则会为生成的张量记录梯度。

torch.stack 是 PyTorch 中的一个函数，用于沿着一个新维度将一组张量拼接在一起。与 torch.cat 不同的是，torch.stack 会在指定的维度上添加一个新的维度，并在该维度上堆叠输入的张量。

torch.stack(tensors, dim=0, out=None)

    tensors：一个可迭代的张量序列（如列表、元组等）。这些张量的形状必须相同。
    dim（可选）：要在其上插入新维度的位置。默认是 0，表示在第一个维度（批次维度）上堆叠张量。
    out（可选）：输出张量。如果指定了这个参数，那么结果会直接写入这个张量。

In [6]:
import torch

# 创建两个形状为 (3, 4) 的张量
a = torch.tensor([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12]])

b = torch.tensor([[13, 14, 15, 16],
                  [17, 18, 19, 20],
                  [21, 22, 23, 24]])

# 在第0个维度上堆叠
stacked_tensor = torch.stack([a, b], dim=0)
print(stacked_tensor)
stacked_tensor = torch.stack([a, b], dim=1)
print(stacked_tensor)

tensor([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],

        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])
tensor([[[ 1,  2,  3,  4],
         [13, 14, 15, 16]],

        [[ 5,  6,  7,  8],
         [17, 18, 19, 20]],

        [[ 9, 10, 11, 12],
         [21, 22, 23, 24]]])


### 估计模型损失
定义了一个用于估计训练和验证集损失的函数estimate_loss，在评估模式下对模型进行评估，然后返回平均损失。

@torch.no_grad() 是一个上下文管理器，用于临时禁止梯度计算。
在某些情况下，如模型评估或推理时，我们不需要计算梯度，因为我们只需要前向传播来获得预测结果。使用 @torch.no_grad() 可以节省内存和计算资源。

model.eval() 将模型设置为评估模式；model.train() 将模型设置为训练模式（training mode）。

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## 定义基于 Bigram 的语言模型

nn.Embedding 层的主要功能是将每个输入的整数索引映射到一个高维向量。

    # 定义一个嵌入层，词汇表大小为 10,000，嵌入向量的维度为 300
    embedding = nn.Embedding(num_embeddings=10000, embedding_dim=300)

    # 输入一个批量大小为 2，序列长度为 4 的索引张量
    input_indices = torch.tensor([[1, 2, 3, 4], [4, 3, 2, 1]])

    # 通过嵌入层将索引映射到嵌入向量
    output = embedding(input_indices)

    print(output.shape)  # 输出形状为 (2, 4, 300)    


view 进行形状调整
    # 假设 logits 是模型的输出，形状为 (B, T, C)
    B, T, C = 2, 3, 4
    logits = torch.randn(B, T, C)

    # 假设 targets 是目标标签，形状为 (B, T)
    targets = torch.randint(0, C, (B, T))

    # 查看原始形状
    print("Original logits shape:", logits.shape)  # Output: (2, 3, 4)
    print("Original targets shape:", targets.shape)  # Output: (2, 3)

    # 调整形状
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)

    # 查看调整后的形状
    print("Reshaped logits shape:", logits.shape)  # Output: (6, 4)
    print("Reshaped targets shape:", targets.shape)  # Output: (6)

交叉熵（Cross-Entropy）是用来衡量两个概率分布之间差异的损失函数。它通常用于分类任务，特别是在多分类任务中非常常见。
    https://blog.csdn.net/weixin_44211968/article/details/123906631

torch.multinomial 可以让你根据模型预测的概率分布，从候选类别中抽取一个。



generate 函数运行过程

idx = torch.tensor([[1, 2, 3]])  # 假设初始输入为序列 [1, 2, 3]

运行 generate 函数5步，生成新token
generated_idx = model.generate(idx, max_new_tokens=5)

第一步：

输入序列 idx 为 [1, 2, 3]。
模型根据当前序列生成 logits，并且我们只关注最后一个时间步的 logits。
通过 softmax 转换为概率分布，从中采样得到一个新 token，假设采样结果为 4。
将 4 追加到序列，得到 [1, 2, 3, 4]。
第二步：

当前序列 idx 为 [1, 2, 3, 4]。
模型再次生成 logits，并采样一个新 token，假设采样结果为 6。
将 6 追加到序列，得到 [1, 2, 3, 4, 6]。
第三步：

当前序列 idx 为 [1, 2, 3, 4, 6]。
模型生成 logits，采样得到新 token，假设为 2。
将 2 追加到序列，得到 [1, 2, 3, 4, 6, 2]。
第四步：

当前序列 idx 为 [1, 2, 3, 4, 6, 2]。
模型生成 logits，采样得到新 token，假设为 9。
将 9 追加到序列，得到 [1, 2, 3, 4, 6, 2, 9]。
第五步：

当前序列 idx 为 [1, 2, 3, 4, 6, 2, 9]。
模型生成 logits，采样得到新 token，假设为 7。
将 7 追加到序列，得到 [1, 2, 3, 4, 6, 2, 9, 7]。
    

In [8]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

torch.optim.AdamW 是 PyTorch 中的一个优化器，用于调整神经网络模型的参数以最小化损失函数。

In [9]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

optimizer.step() 是 PyTorch 中的一个方法，用于执行优化器的参数更新步骤。它基于当前计算出的梯度，按照优化算法的规则更新模型的参数。

In [10]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.6095, val loss 4.5949
step 300: train loss 2.7809, val loss 2.8023
step 600: train loss 2.5394, val loss 2.5500
step 900: train loss 2.5028, val loss 2.5226
step 1200: train loss 2.4807, val loss 2.5057
step 1500: train loss 2.4731, val loss 2.4894
step 1800: train loss 2.4700, val loss 2.4961
step 2100: train loss 2.4621, val loss 2.4883
step 2400: train loss 2.4611, val loss 2.4919
step 2700: train loss 2.4565, val loss 2.4870

Ourt weffouts I ss,

INoty my t phthoun.
end t win?

Tor y no woinste gr pithBuinores:

GEde hero wiungie'RETons!
Hathe! t no thanesh.
Thy manen t ind int' thie NUStegignathanangth veicemothas ghithemomes ie. trt litat,
INRK:

Bll accad.
GLUExivesw,-n geavin m ththe'e'?

A:

Fio my ntheare ds y
hawon;
NI o CHe,
USeremed gse d he
A ulllshe the annow, wiche-NV: ge t k; pr'get pll--be by; iA ts cthe 'ves ly,
Kithe byo aie nofd n is toupome,Tenthist!


ARou.
On h!

Hat,
Iffughomabbeeke nd ber.
A:
Ma
